In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
from pycaret.classification import *
from pycaret.classification import *
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
import function_script

In [2]:
if __name__ == "__main__":
    #Parameter
    target_csv  = 'abalone'#['abalone','50_50', 'fill']
    ground_truth = 'Sex' #['Sex', 'Diabetes_binary']
    Prediction_label = 'All' #['M_F', 'All'] 
    display_bool = False
    rand_seed = 42
    #1.1 Load ============================================================
    if target_csv == 'fill':
        df = pd.read_csv('../data/diabetes_binary_health_indicators_BRFSS2015.csv')
    if target_csv == '50_50':
        df = pd.read_csv('../data/diabetes_binary_5050split_health_indicators_BRFSS2015.csv')
    if target_csv == 'abalone':
        df = pd.read_csv('../data/abalone.csv')

    #2.1 Load Label ['M_F', 'All']  ============================================================        
    if target_csv == 'abalone' and Prediction_label == 'M_F':
        df = df[df['Sex'] != 'I']
    print(df)

    # Polynomial Features for interactions (degree 2 for example)
    poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
    continuous_columns = ['Length', 'Diameter', 'Height', 'Whole_weight', 'Shucked_weight', 'Viscera_weight', 'Shell_weight']
    df_poly = poly.fit_transform(df[continuous_columns])
    poly_features = poly.get_feature_names_out(continuous_columns)  # Updated line here
    df_poly = pd.DataFrame(df_poly, columns=poly_features)
    
    # Concatenate the new polynomial features to the original dataframe
    df = pd.concat([df, df_poly], axis=1)
    
    # Feature Selection
    # Since we now have many features, we can use SelectKBest to choose the top k features that have the strongest relationships with the target variable
    selector = SelectKBest(f_classif, k=10)  # Let's assume we want to keep the top 10 features for now
    X_new = selector.fit_transform(df.drop('Sex', axis=1), df['Sex'])
    selected_features = df.drop('Sex', axis=1).columns[selector.get_support()]
    
    # Now create a new DataFrame with selected features
    df= df[selected_features]
    print(df)

    #3.1 Machine learning ============================================================
    #3.2 Splitting 60/20/20
   
    train_val_data, test_data = train_test_split(df, test_size=0.2, random_state=rand_seed) #split 80/20
    print(train_val_data.shape, test_data.shape)

     Sex  Length  Diameter  Height  Whole_weight  Shucked_weight  \
0      M   0.455     0.365   0.095        0.5140          0.2245   
1      M   0.350     0.265   0.090        0.2255          0.0995   
2      F   0.530     0.420   0.135        0.6770          0.2565   
3      M   0.440     0.365   0.125        0.5160          0.2155   
4      I   0.330     0.255   0.080        0.2050          0.0895   
...   ..     ...       ...     ...           ...             ...   
4172   F   0.565     0.450   0.165        0.8870          0.3700   
4173   M   0.590     0.440   0.135        0.9660          0.4390   
4174   M   0.600     0.475   0.205        1.1760          0.5255   
4175   F   0.625     0.485   0.150        1.0945          0.5310   
4176   M   0.710     0.555   0.195        1.9485          0.9455   

      Viscera_weight  Shell_weight  Rings  
0             0.1010        0.1500     15  
1             0.0485        0.0700      7  
2             0.1415        0.2100      9  
3      